# LSTM

데모용으로 올려둠 

하지만 정확도가 처참하다. 나중에는 고도화. 

In [2]:
#!pip install ccxt 
#!pip install pandas-ta 
#!pip install optuna

In [30]:
import pandas as pd
import ccxt 
import numpy as np 
import pandas as pd 
from tqdm.auto import tqdm 
from xgboost import XGBClassifier
import json 
import pandas_ta as ta
from sklearn.metrics import accuracy_score 

import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split 
import zoneinfo

In [14]:
chart_df = pd.read_feather('BTC_USDT-1d (1).feather')
chart_df['date'] = pd.to_datetime(chart_df['date'])
# Extract year, month, day, and time information
chart_df['month'] = chart_df['date'].dt.month
chart_df['day'] = chart_df['date'].dt.day
chart_df.drop(columns={"date"}, inplace=True) 

In [26]:
chart_df

,open,high,low,close,volume,month,day,targets
0,4261.48,4485.39,4200.74,4285.08,795.150377,8,17,0.0
1,4285.08,4371.52,3938.77,4108.37,1199.888264,8,18,1.0
2,4108.37,4184.69,3850.00,4139.98,381.309763,8,19,0.0
3,4120.98,4211.08,4032.62,4086.29,467.083022,8,20,0.0
4,4069.13,4119.62,3911.79,4016.00,691.743060,8,21,1.0
...,...,...,...,...,...,...,...,...
2295,37818.88,38450.00,37570.00,37854.64,32994.191070,11,29,0.0
2296,37854.65,38145.85,37500.00,37723.96,24740.291470,11,30,1.0
2297,37723.97,38999.00,37615.86,38682.52,43415.663240,12,1,1.0
2298,38682.51,39717.14,38641.61,39450.35,26696.921610,12,2,1.0


In [15]:
close = chart_df["close"].values 

targets = [] 
for i in range(len(close)-1): 
    diff = (close[i+1] - close[i]) / close[i] 
    if diff < 0:
        targets.append(0) # short 
    else: 
        targets.append(1) # long 
        
targets.append(None) 

In [16]:
chart_df["targets"] = targets

chart_df.dropna(inplace=True) 

In [17]:
scaler = MinMaxScaler() 
scaled_features = scaler.fit_transform(chart_df.drop('targets', axis=1)) 

In [18]:
def create_dataset(X, y, time_step=1): 
    Xs, ys = [], [] 
    for i in range(len(X) - time_steps): 
        v = X.iloc[i:(i + time_steps)].values 
        Xs.append(v) 
        ys.append(y.iloc[i + time_steps]) 
    return np.array(Xs), np.array(ys) 

In [19]:
time_steps = 14
X, y = create_dataset(pd.DataFrame(scaled_features), chart_df['targets'], time_steps)

train_size = int(X.shape[0] * 0.8)

X_train = X[:train_size] 
y_train = y[:train_size] 

X_val = X[train_size:] 
y_val = y[train_size:] 

In [20]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=int)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=int)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

(torch.Size([1828, 14, 7]),
 torch.Size([1828]),
 torch.Size([458, 14, 7]),
 torch.Size([458]))

In [21]:
batch_size = 32
train_data = TensorDataset(X_train, y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(X_val, y_val) 
val_sampler = SequentialSampler(val_data) 
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size) 

In [22]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
    def forward(self, input_seq):
        # Initialize hidden and cell states with zeros
        h0 = torch.zeros(1, input_seq.size(0), self.hidden_layer_size).to(input_seq.device)
        c0 = torch.zeros(1, input_seq.size(0), self.hidden_layer_size).to(input_seq.device)
        lstm_out, _ = self.lstm(input_seq, (h0, c0))
        last_time_step = lstm_out[:, -1, :]
        predictions = self.linear(last_time_step)
        return predictions

In [23]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
device = torch.device("cuda") 

input_size = 7
hidden_layer_size = 64
output_size = 2
epochs = 10 
model = LSTMClassifier(input_size, hidden_layer_size, output_size)  
model.to(device) 

device = torch.device("cuda:0")

loss_func = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

best_val_loss = np.inf

model.zero_grad()
for epoch in tqdm(range(epochs), desc="epochs", position=0, leave=True): 
    train_loss = 0 
    model.train() 
    for step, batch in enumerate(train_dataloader): 
        batch = tuple(t.to(device) for t in batch) 
        b_x, b_y = batch 
        output = model(b_x)
        loss = loss_func(output, b_y) 
        train_loss += loss.item() 
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 
        optimizer.step() 
        model.zero_grad() 
    
    val_loss = 0 
    val_accuracy = 0 
    model.eval() 
    for step, batch in enumerate(val_dataloader):
        batch = tuple(t.to(device) for t in batch) 
        b_x, b_y = batch 
        with torch.no_grad():
            output = model(b_x) 
            loss = loss_func(output, b_y) 
            val_loss += loss.item() 
            val_accuracy += flat_accuracy(output.detach().cpu().numpy(), b_y.detach().cpu().numpy()) 
    
    avg_train_loss = train_loss / len(train_dataloader) 
    avg_val_loss = val_loss / len(val_dataloader) 
    avg_val_accuracy = val_accuracy / len(val_dataloader)
    if best_val_loss > avg_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "best_lstm.pt")
    
    print(f"Epoch:{epoch} | avg train loss:{avg_train_loss} | avg_val_loss:{avg_val_loss} | avg_val_accuracy:{avg_val_accuracy}")

epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch:0 | avg train loss:0.6951619047543098 | avg_val_loss:0.6931251009305318 | avg_val_accuracy:0.5033333333333333
Epoch:1 | avg train loss:0.6934267097506029 | avg_val_loss:0.6930562535921733 | avg_val_accuracy:0.5320833333333334
Epoch:2 | avg train loss:0.6927043261199162 | avg_val_loss:0.6931762218475341 | avg_val_accuracy:0.5091666666666667
Epoch:3 | avg train loss:0.692384038505883 | avg_val_loss:0.6935765663782756 | avg_val_accuracy:0.4966666666666667
Epoch:4 | avg train loss:0.6916799103391582 | avg_val_loss:0.6936832507451375 | avg_val_accuracy:0.4966666666666667
Epoch:5 | avg train loss:0.6920113409387654 | avg_val_loss:0.6939559499422709 | avg_val_accuracy:0.4966666666666667
Epoch:6 | avg train loss:0.6919664360325912 | avg_val_loss:0.6938185254732768 | avg_val_accuracy:0.4966666666666667
Epoch:7 | avg train loss:0.6924109489753328 | avg_val_loss:0.6940711696942647 | avg_val_accuracy:0.4966666666666667
Epoch:8 | avg train loss:0.692068338394165 | avg_val_loss:0.6937520583470

In [39]:
# save scaler 
import joblib 
joblib.dump(scaler, 'LSTM_minmax_scaler.pkl')

['LSTM_minmax_scaler.pkl']

# Inference Code

In [25]:
# run inference on CPU 
input_size = 7
hidden_layer_size = 64
output_size = 2
LSTM_test = LSTMClassifier(input_size, hidden_layer_size, output_size)  
checkpoint = torch.load("best_lstm.pt") 
LSTM_test.load_state_dict(checkpoint)

<All keys matched successfully>

In [40]:
test_scaler = joblib.load("LSTM_minmax_scaler.pkl")

In [28]:
def preprocess(df):
    # Convert timestamps to datetime objects in Korean timezone
    korean_timezone = zoneinfo.ZoneInfo("Asia/Seoul")
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    df['datetime'] = df['datetime'].dt.tz_convert(korean_timezone)
    return df

In [46]:
bitget = ccxt.bitget()
ohlcv = bitget.fetch_ohlcv("BTC/USDT:USDT", "1d")
chart_df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
chart_df = preprocess(chart_df)
days, months = [], []
for dt in chart_df["datetime"]:
    dtobj = pd.to_datetime(dt)
    day = dtobj.day
    month = dtobj.month
    days.append(day)
    months.append(month)
chart_df["month"] = months
chart_df["day"] = days
chart_df.drop(columns={"timestamp", "datetime"}, inplace=True)

In [43]:
test_scaled_features = test_scaler.transform(chart_df)  

In [59]:
test_input = test_scaled_features[-15:-1]  
test_input = test_input.reshape((-1, 14, 7)) 
test_input = torch.tensor(test_input, dtype=torch.float32)

LSTM_test.eval()
with torch.no_grad():
    output = LSTM_test(test_input) 
    probs = nn.Softmax()(output)[0] 
    print(f"long prob: {probs[1]}%")  
    print(f"short prob: {probs[0]}%") 

long prob: 0.5021270513534546%
short prob: 0.49787288904190063%


/tmp/ipykernel_571/2476712823.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = nn.Softmax()(output)[0]


In [68]:
# inference function 
def infer_LSTM(): 
    input_size = 7
    hidden_layer_size = 64
    output_size = 2
    LSTM_test = LSTMClassifier(input_size, hidden_layer_size, output_size)  
    checkpoint = torch.load("best_lstm.pt") 
    LSTM_test.load_state_dict(checkpoint)
    test_scaler = joblib.load("LSTM_minmax_scaler.pkl")
    
    bitget = ccxt.bitget()
    ohlcv = bitget.fetch_ohlcv("BTC/USDT:USDT", "1d")
    chart_df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    chart_df = preprocess(chart_df)
    days, months = [], []
    for dt in chart_df["datetime"]:
        dtobj = pd.to_datetime(dt)
        day = dtobj.day
        month = dtobj.month
        days.append(day)
        months.append(month)
    chart_df["month"] = months
    chart_df["day"] = days
    chart_df.drop(columns={"timestamp", "datetime"}, inplace=True)
    
    test_scaled_features = test_scaler.transform(chart_df)  
    test_input = test_scaled_features[-15:-1]  
    test_input = test_input.reshape((-1, 14, 7)) 
    test_input = torch.tensor(test_input, dtype=torch.float32)

    LSTM_test.eval()
    with torch.no_grad():
        output = LSTM_test(test_input) 
        probs = nn.Softmax(dim=-1)(output)[0] 
        probs = probs.detach().cpu().numpy() 

    LSTM_long_prob = round(probs[1] * 100, 2)
    LSTM_short_prob = round(probs[0] * 100, 2)
    return LSTM_long_prob, LSTM_short_prob

In [69]:
infer_LSTM()

(50.21, 49.79)